In [1]:
import pandas as pd

In [119]:
df_1 = pd.read_csv("/tf/Classification/Data/BRCA/Using_cbioportal/data_RNA_Seq_v2_expression_median.tsv", sep = '\t')

In [120]:
df_1.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-3C-AAAU-01,TCGA-3C-AALI-01,TCGA-3C-AALJ-01,TCGA-3C-AALK-01,TCGA-4H-AAAK-01,TCGA-5L-AAT0-01,TCGA-5L-AAT1-01,TCGA-5T-A9QA-01,...,TCGA-UL-AAZ6-01,TCGA-UU-A93S-01,TCGA-V7-A7HQ-01,TCGA-W8-A86G-01,TCGA-WT-AB41-01,TCGA-WT-AB44-01,TCGA-XX-A899-01,TCGA-XX-A89A-01,TCGA-Z7-A8R5-01,TCGA-Z7-A8R6-01
0,LOC100130426,100130426,0.0000,0.0000,0.9066,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,UBE2Q2P3,100133144,16.3644,9.2659,11.6228,12.0894,6.8468,3.9889,0.0000,1.4644,...,0.3992,4.3126,0.0000,5.5624,0.0000,0.0000,14.3858,22.3240,2.2638,6.8865
2,UBE2Q2P3,100134869,12.9316,17.3790,9.2294,11.0799,14.4298,13.6090,10.5949,8.9958,...,14.3720,10.8828,3.0792,14.3711,6.3091,3.2580,21.4409,27.2744,7.2933,24.7795
3,LOC149767,10357,52.1503,69.7553,154.2974,143.8643,84.2128,114.2572,115.9984,107.5628,...,135.6241,136.1288,29.9974,128.3151,53.6278,42.2643,137.7756,64.1427,85.0461,167.5511
4,TIMM23,10431,408.0760,563.8934,1360.8341,865.5358,766.3830,807.7431,1108.3945,1420.5021,...,1570.1445,2886.3965,1721.8816,697.6744,1245.2681,1877.4180,652.7559,722.7208,1140.2801,1003.5668


In [121]:
df_2 = pd.read_csv("/tf/Classification/Data/BRCA/Using_cbioportal/brca_tcga_pan_can_atlas_clinical_data.tsv", sep = '\t')

In [122]:
df_2.head()

,Study ID,Patient ID,Sample ID,Diagnosis Age,Neoplasm Disease Stage American Joint Committee on Cancer Code,American Joint Committee on Cancer Publication Version Type,Aneuploidy Score,Buffa Hypoxia Score,Cancer Type,TCGA PanCanAtlas Cancer Type Acronym,...,9p Status,9q Status,Subtype,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site,Tumor Disease Anatomic Site,Tumor Type,Patient Weight,Winter Hypoxia Score
0,brca_tcga_pan_can_atlas_2018,TCGA-3C-AAAU,TCGA-3C-AAAU-01,55,STAGE X,6TH,19.0,-21.0,Invasive Breast Carcinoma,BRCA,...,Lost,Lost,BRCA_LumA,No,Yes,3C,Breast,Infiltrating Lobular Carcinoma,NaN,-28.0
1,brca_tcga_pan_can_atlas_2018,TCGA-3C-AALI,TCGA-3C-AALI-01,50,STAGE IIB,6TH,22.0,5.0,Invasive Breast Carcinoma,BRCA,...,Lost,NaN,BRCA_Her2,No,Yes,3C,Breast,Infiltrating Ductal Carcinoma,NaN,20.0
2,brca_tcga_pan_can_atlas_2018,TCGA-3C-AALJ,TCGA-3C-AALJ-01,62,STAGE IIB,7TH,13.0,-5.0,Invasive Breast Carcinoma,BRCA,...,Not Called,Gained,BRCA_LumB,No,Yes,3C,Breast,Infiltrating Ductal Carcinoma,NaN,-10.0
3,brca_tcga_pan_can_atlas_2018,TCGA-3C-AALK,TCGA-3C-AALK-01,52,STAGE IA,7TH,4.0,-27.0,Invasive Breast Carcinoma,BRCA,...,Not Called,Not Called,BRCA_LumA,No,Yes,3C,Breast,Infiltrating Ductal Carcinoma,NaN,4.0
4,brca_tcga_pan_can_atlas_2018,TCGA-4H-AAAK,TCGA-4H-AAAK-01,50,STAGE IIIA,7TH,7.0,-27.0,Invasive Breast Carcinoma,BRCA,...,Gained,Gained,BRCA_LumA,Yes,No,4H,Breast,Infiltrating Lobular Carcinoma,NaN,-20.0


In [134]:
Y = df_2[['Patient ID', 'Subtype']]
Y.head()

,Patient ID,Subtype
0,TCGA-3C-AAAU,BRCA_LumA
1,TCGA-3C-AALI,BRCA_Her2
2,TCGA-3C-AALJ,BRCA_LumB
3,TCGA-3C-AALK,BRCA_LumA
4,TCGA-4H-AAAK,BRCA_LumA


In [124]:
Y_patient_code = Y['Patient ID'].tolist()
Y_patient_code

['TCGA-3C-AAAU',
 'TCGA-3C-AALI',
 'TCGA-3C-AALJ',
 'TCGA-3C-AALK',
 'TCGA-4H-AAAK',
 'TCGA-5L-AAT0',
 'TCGA-5T-A9QA',
 'TCGA-A1-A0SB',
 'TCGA-A1-A0SD',
 'TCGA-A1-A0SE',
 'TCGA-A1-A0SF',
 'TCGA-A1-A0SG',
 'TCGA-A1-A0SH',
 'TCGA-A1-A0SI',
 'TCGA-A1-A0SJ',
 'TCGA-A1-A0SK',
 'TCGA-A1-A0SM',
 'TCGA-A1-A0SN',
 'TCGA-A1-A0SO',
 'TCGA-A1-A0SP',
 'TCGA-A1-A0SQ',
 'TCGA-A2-A04N',
 'TCGA-A2-A04P',
 'TCGA-A2-A04Q',
 'TCGA-A2-A04R',
 'TCGA-A2-A04T',
 'TCGA-A2-A04U',
 'TCGA-A2-A04V',
 'TCGA-A2-A04W',
 'TCGA-A2-A04X',
 'TCGA-A2-A04Y',
 'TCGA-A2-A0CK',
 'TCGA-A2-A0CL',
 'TCGA-A2-A0CM',
 'TCGA-A2-A0CO',
 'TCGA-A2-A0CP',
 'TCGA-A2-A0CQ',
 'TCGA-A2-A0CR',
 'TCGA-A2-A0CS',
 'TCGA-A2-A0CT',
 'TCGA-A2-A0CU',
 'TCGA-A2-A0CV',
 'TCGA-A2-A0CW',
 'TCGA-A2-A0CX',
 'TCGA-A2-A0CZ',
 'TCGA-A2-A0D0',
 'TCGA-A2-A0D1',
 'TCGA-A2-A0D2',
 'TCGA-A2-A0D3',
 'TCGA-A2-A0D4',
 'TCGA-A2-A0EM',
 'TCGA-A2-A0EN',
 'TCGA-A2-A0EO',
 'TCGA-A2-A0EP',
 'TCGA-A2-A0EQ',
 'TCGA-A2-A0ER',
 'TCGA-A2-A0ES',
 'TCGA-A2-A0ET',
 'TCGA-A2-A0EU

In [125]:
df_patient_code = df_1.columns[2:]
df_patient_code = [x[:-3] for x in df_patient_code]
df_patient_code

['TCGA-3C-AAAU',
 'TCGA-3C-AALI',
 'TCGA-3C-AALJ',
 'TCGA-3C-AALK',
 'TCGA-4H-AAAK',
 'TCGA-5L-AAT0',
 'TCGA-5L-AAT1',
 'TCGA-5T-A9QA',
 'TCGA-A1-A0SB',
 'TCGA-A1-A0SD',
 'TCGA-A1-A0SE',
 'TCGA-A1-A0SF',
 'TCGA-A1-A0SG',
 'TCGA-A1-A0SH',
 'TCGA-A1-A0SI',
 'TCGA-A1-A0SJ',
 'TCGA-A1-A0SK',
 'TCGA-A1-A0SM',
 'TCGA-A1-A0SN',
 'TCGA-A1-A0SO',
 'TCGA-A1-A0SP',
 'TCGA-A1-A0SQ',
 'TCGA-A2-A04N',
 'TCGA-A2-A04P',
 'TCGA-A2-A04Q',
 'TCGA-A2-A04R',
 'TCGA-A2-A04T',
 'TCGA-A2-A04U',
 'TCGA-A2-A04V',
 'TCGA-A2-A04W',
 'TCGA-A2-A04X',
 'TCGA-A2-A04Y',
 'TCGA-A2-A0CK',
 'TCGA-A2-A0CL',
 'TCGA-A2-A0CM',
 'TCGA-A2-A0CO',
 'TCGA-A2-A0CP',
 'TCGA-A2-A0CQ',
 'TCGA-A2-A0CR',
 'TCGA-A2-A0CS',
 'TCGA-A2-A0CT',
 'TCGA-A2-A0CU',
 'TCGA-A2-A0CV',
 'TCGA-A2-A0CW',
 'TCGA-A2-A0CX',
 'TCGA-A2-A0CY',
 'TCGA-A2-A0CZ',
 'TCGA-A2-A0D0',
 'TCGA-A2-A0D1',
 'TCGA-A2-A0D2',
 'TCGA-A2-A0D3',
 'TCGA-A2-A0D4',
 'TCGA-A2-A0EM',
 'TCGA-A2-A0EN',
 'TCGA-A2-A0EO',
 'TCGA-A2-A0EP',
 'TCGA-A2-A0EQ',
 'TCGA-A2-A0ER',
 'TCGA-A2-A0ES

In [126]:
ans = [x for x in df_patient_code if x in Y_patient_code]   ## patient ID's that do have PAM50 avaialable

In [127]:
len(ans)

1086

In [128]:
len(Y_patient_code)    ## Number of patient ID's with PAM50 values in df_2

1084

In [129]:
len(df_patient_code)   ## Total Number of patient ID's in df_1

1100

In [130]:
len(set(df_patient_code))   ## Unique patient ID's in df_1

1093

In [131]:
len(set(Y_patient_code))   ## Unique patient Id's with PAM50 in df_2

1084

In [132]:
len(set(ans))     ## Unique patients in df_1 dataset with their PAM_50 in df_2 dataset

1079

In [135]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [154]:
## Prepare 2 sets S and C
## S initially contains only the bias term and incrementally adds the features
## C set contains all other features which are not contained in S set
## Design a neural network with ReLu as non linear activation

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K

In [156]:
## Input : x - 2D tensor of (number of samples (N) X Number of features (F))
SUBTYPES = 5

def build_a_subnetwork(x, y):
    
    input_size = x.shape[1]  ## number of features in input
    
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(input_size,), kernel_regularizer=keras.regularizers.l2(0.01)),  # kernel regulariser are applied to weights
      tf.keras.layers.Dense(10, activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l2(0.01)),  ## as per research paper we have to take lp regularisation where p = 2
      tf.keras.layers.Dense(SUBTYPES)
    ])
    
    return model

# build_a_subnetwork(tf.constant([[1],[2]]), tf.constant([1,0]))

In [ ]:
def train_a_network(model, x, y):
    
    K.set_value(model.optimizer.learning_rate, 0.001)  ##  initial value of learning rate
    model.compile(optimizer="Adagrad", loss="CategoricalCrossentropy", metrics=["mae", "acc"])
    model.fit(x, y)